In [1]:
from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np

ctrl = Controller(*init_config())

2021-12-30 22:48:49.289621: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir
Namespace(act='elu', action_gnn_hidden_act=64, action_gnn_hidden_val=200, action_gnn_layers=6, action_gnn_next_shape=(32, 64, 32), action_gnn_start_shape=(4, 1, 4), action_repeat=1, actor_disc=5, actor_dist='normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=0.001, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, atari_grayscale=False, behavior_stop_grad=True, clip_rewards='identity', collect_dyn_sample=True, dataset_size=100000, debug=False, disag_layers=1, disag_log=True, disag_models=3, disag_offset=0, disag_target='embed', disag_units=200, discount=0.95, discount_lambda=0.8, discount_layers=2, discount_scale=1.0, dyn_cell='gru', dyn_deter=512, dyn_discrete=32, dyn_hidden=512, dyn_input_layers=5, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_lay

usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir
2021-12-30 22:48:50.743013: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-30 22:48:50.743667: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-30 22:48:50.779060: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-30 22:48:50.779718: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemor

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t6_orders_2.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t41_zfmisc_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Prefill dataset (0 steps).
Loaded problem: m2n140__t16_tops_1
Eval episode has 11 steps and return -0.200.


/home/kristsz/DreamProLog/envs/wrappers.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  value = np.array(value)


In [2]:
meta = ctrl.datasetManager._train_eps._meta
ds = []
for v in meta.values():
    ds.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    return np.pad(narr, [0, 128-size])

parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [3]:
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    for i in range(1000):
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x, x
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

2021-12-30 22:49:01.384666: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-30 22:49:01.599230: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-30 22:49:01.599829: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-30 22:49:01.599851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] 

In [4]:
from unorganized.parts2 import Net

GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [5]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.0):
        super().__init__()
        self.omega = omega
        
    def call(self, y_true, y_pred):
        y = tf.nn.softmax(y_pred, axis=-1)
        y_probs = tf.gather(y, y_true, axis=-1, batch_dims=2)
        y_log_probs = tf.math.log(y_probs+0.001)
        #loss = self.omega*y_log_probs + (1-self.omega)*tf.math.cumsum(y_log_probs, axis=-1)
        #loss = y_log_probs*tf.math.cumprod(0.97*tf.ones((32,128)), axis=-1)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        #mask = tf.math.cumprod(mask, axis=-1)
        loss = (mask+self.omega)*y_log_probs
        loss = -tf.math.reduce_sum(loss)/(128*32)
        return loss
    
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.0):
        super().__init__()
        self.omega = omega
        self.loss_objective = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        
    def call(self, y_true, y_pred):
        loss = self.loss_objective(y_true, y_pred)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        
        loss *= mask
        return loss

In [6]:
model = Net(1)#Net(1, 8, 128, 64, 4, 128)#Net(128, 4, 256)
#d_model = 256
#enc_embed = tf.keras.layers.Embedding(300, d_model)
#encoder = Encoder(4, d_model, 4, 512, 0.04)

In [7]:
x, x = next(iter(tf_ds))
model(x, False)
y_true = x
mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
mask = tf.math.cumprod(mask, axis=-1)

2021-12-30 22:49:02.391966: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-30 22:49:02.409409: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3598190000 Hz
2021-12-30 22:49:02.433723: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-30 22:49:02.607983: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=Loss(), metrics=[])


In [9]:
model.fit(tf_ds, epochs = 50)

Epoch 1/50
1000/1000 [==============================] - 11s 9ms/step - loss: 0.7101
Epoch 2/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.5241
Epoch 3/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4740
Epoch 4/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4376
Epoch 5/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4214
Epoch 6/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4070
Epoch 7/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4019
Epoch 8/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3982
Epoch 9/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3871
Epoch 10/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3815
Epoch 11/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3767
Epoch 12/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3750


In [21]:

x, y_true = next(iter(tf_ds))
y_pred = model(x, False)
y_pred = tf.nn.softmax(y_pred, axis=-1)
y_probs = tf.gather(y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(y_pred, axis=-1)
print(x[:4], y_pred[:4], tf.cast(100*y_probs[:4], dtype=tf.int32), sep='\n\n')

tf.Tensor(
[[ 21   5  12  22  23  24   5  13  16   2   9   5  16  40   2 109  20  12
   13  16   2   3   9   5  12  40   2   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [ 21   5  12  22  23  24   5  13  16   2   3  27  23 125  20  13  16  12
    2  27  23  12   2 109  20  12  13  16   2   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   

In [23]:
model.encoder.variable

<tf.Variable 'Variable:0' shape=(1, 8, 128) dtype=float32, numpy=
array([[[ 0.70087147,  0.1958967 , -0.35145903, ...,  0.48064098,
          0.4918324 ,  0.03355688],
        [ 0.20909105,  0.69546527, -0.23883833, ...,  0.5814026 ,
          0.25828016,  0.5053409 ],
        [-0.21123317,  0.7534062 ,  0.7333148 , ..., -0.10376038,
          0.6470314 , -0.12154149],
        ...,
        [ 0.46920526, -0.04690957,  0.6305629 , ...,  0.34775922,
          0.04588863, -0.03653677],
        [ 0.03515827, -0.09518815,  0.4213055 , ...,  0.26995596,
          0.2315686 , -0.09755877],
        [ 0.15336037,  0.21321705,  0.70800805, ...,  0.54270494,
         -0.08866469,  0.643626  ]]], dtype=float32)>

In [12]:
mask = tf.cast(tf.math.equal(x[:1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        
v, q = model.enc_embed(x)[:1], model.encoder.variable#tf.tile(model.encoder.variable, (32, 1, 1))
v = [v]
q = [q]
attention = []
for l in model.encoder.layers:
    _v, _q, _att = l(v[-1], q[-1], mask, False)
    v.append(_v)
    q.append(_q)
    attention.append(_att)
    

In [13]:
v

[<tf.Tensor: shape=(1, 128, 128), dtype=float32, numpy=
 array([[[ 0.04792119,  0.04437996, -0.00787528, ..., -0.07554483,
           0.31612352, -0.24282289],
         [-0.23656398,  0.00266965, -0.15428829, ..., -0.26722905,
           0.05645089,  0.11960076],
         [ 0.19339246, -0.05289397,  0.11940589, ...,  0.32167187,
          -0.48939654,  0.04903906],
         ...,
         [-0.49084064,  0.65793324, -0.6095724 , ..., -0.9494842 ,
           0.08307586,  0.10185409],
         [-0.49084064,  0.65793324, -0.6095724 , ..., -0.9494842 ,
           0.08307586,  0.10185409],
         [-0.49084064,  0.65793324, -0.6095724 , ..., -0.9494842 ,
           0.08307586,  0.10185409]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 128, 128), dtype=float32, numpy=
 array([[[ 0.11194344, -0.0637473 , -0.8052465 , ..., -0.12497996,
           0.51567215,  0.29573622],
         [ 0.3811726 , -0.05803382,  0.00395416, ...,  0.14346313,
          -0.2781397 ,  0.24369061],
         [ 0.4371124 , 

In [14]:
attention[0]

(<tf.Tensor: shape=(1, 4, 128, 8), dtype=float32, numpy=
 array([[[[2.51413316e-01, 1.04385642e-02, 7.05200341e-03, ...,
           1.28090337e-01, 5.06249487e-01, 8.14314261e-02],
          [6.94326081e-06, 2.74363818e-04, 1.09457616e-02, ...,
           3.74753581e-05, 4.39365249e-05, 1.62385895e-05],
          [5.51492907e-03, 1.52532076e-02, 9.39111948e-01, ...,
           3.09610344e-03, 2.06962833e-03, 7.02536013e-03],
          ...,
          [1.60381082e-28, 1.60381082e-28, 1.60381082e-28, ...,
           1.60381082e-28, 1.60381082e-28, 1.60381082e-28],
          [1.60381082e-28, 1.60381082e-28, 1.60381082e-28, ...,
           1.60381082e-28, 1.60381082e-28, 1.60381082e-28],
          [1.60381082e-28, 1.60381082e-28, 1.60381082e-28, ...,
           1.60381082e-28, 1.60381082e-28, 1.60381082e-28]],
 
         [[9.34151001e-03, 1.08260974e-01, 1.45780921e-01, ...,
           7.30265602e-02, 3.63879353e-02, 1.66411586e-02],
          [1.56238035e-04, 1.19090881e-02, 8.00474510e-02

In [15]:
q

[<tf.Variable 'Variable:0' shape=(1, 8, 128) dtype=float32, numpy=
 array([[[ 0.70087147,  0.1958967 , -0.35145903, ...,  0.48064098,
           0.4918324 ,  0.03355688],
         [ 0.20909105,  0.69546527, -0.23883833, ...,  0.5814026 ,
           0.25828016,  0.5053409 ],
         [-0.21123317,  0.7534062 ,  0.7333148 , ..., -0.10376038,
           0.6470314 , -0.12154149],
         ...,
         [ 0.46920526, -0.04690957,  0.6305629 , ...,  0.34775922,
           0.04588863, -0.03653677],
         [ 0.03515827, -0.09518815,  0.4213055 , ...,  0.26995596,
           0.2315686 , -0.09755877],
         [ 0.15336037,  0.21321705,  0.70800805, ...,  0.54270494,
          -0.08866469,  0.643626  ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 8, 128), dtype=float32, numpy=
 array([[[-8.9805186e-02, -1.5886374e-01,  9.1277137e-02, ...,
           3.6378831e-01,  2.5522321e-02,  3.0075678e-01],
         [-8.1214778e-02, -1.6048390e-01,  9.6112058e-02, ...,
           3.1804240e-01,  2.0409845e

In [16]:
tf.ones((32, 4, 128, 128))

<tf.Tensor: shape=(32, 4, 128, 128), dtype=float32, numpy=
array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],

In [17]:
def eigenRank(tensor):
    batch_shape = tf.shape(tensor)[:-2]
    matrix_shape = tf.shape(tensor)[-2:]
    l = len(batch_shape)
    print(tensor.shape)
    if matrix_shape[0]>matrix_shape[1]: tensor = tf.transpose(tensor, tuple(i for i in range(l)) + (l+1, l))
    M = tf.matmul(tensor, tensor, transpose_b = True)
    print(tensor.shape, M.shape)
    trace = tf.linalg.trace(M)
    M_squared = tf.matmul(M, M)
    print(M_squared.shape)
    trace_squared = tf.linalg.trace(M_squared)
    return (trace**2)/trace_squared

In [18]:
att = attention[0][0]

In [19]:
eigenRank(tf.random.uniform(
            (4,8, 128, 128), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=420, name=None))

(4, 8, 128, 128)
(4, 8, 128, 128) (4, 8, 128, 128)
(4, 8, 128, 128)


<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[1.7550379, 1.7803738, 1.75285  , 1.7513034, 1.7525351, 1.7638059,
        1.7347128, 1.7705356],
       [1.7289912, 1.7548829, 1.7417741, 1.7558697, 1.7845999, 1.7643174,
        1.756217 , 1.7740963],
       [1.7501621, 1.7563052, 1.7588693, 1.7585539, 1.7645165, 1.745992 ,
        1.7515677, 1.7681904],
       [1.7503204, 1.7486261, 1.7505194, 1.763513 , 1.7547823, 1.742856 ,
        1.7614138, 1.7686299]], dtype=float32)>

In [20]:
mask = tf.cast(tf.math.equal(x[:1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        
v = model.enc_embed(x)[:]#, model.encoder.variable#tf.tile(model.encoder.variable, (32, 1, 1))
vs = [v]
att = []
for mha, sl in zip(model.mha_layers, model.sl_layers):
    v, a = mha(v, v, v, mask)
    v = sl(v)
    vs.append(v)
    att.append(a)

AttributeError: 'Net' object has no attribute 'mha_layers'

In [ ]:
z = vs[-1][:8, :1, :16]
z

In [ ]:
z = z - z[:, :1, :]

In [ ]:
tf.reduce_sum(tf.cast(z>0.001, tf.float32))

In [ ]:
att[0][:1, :1, :16, :16]

In [ ]:
import tensorflow_probability as tfp

normal = tfp.distributions.Normal(tf.zeros((32)),  tf.ones((32)))
x = tf.cast(tf.math.softmax(normal.sample(24)*tf.constant([[(i+1)*0.4] for i in range(24)]), axis=-1)>0.1, tf.int32)
